In [4]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers

In [5]:
from tensorflow.keras.models import load_model

In [94]:
interpreter = tf.lite.Interpreter(model_path='new/InceptionV3.tflite')
interpreter.allocate_tensors()

In [95]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [96]:
print("Input shape:", input_details[0]['shape'])
print("Input type:", input_details[0]['dtype'])
print("Output shape:", output_details[0]['shape'])
print("Output type:", output_details[0]['dtype'])
input_shape = input_details[0]['shape']
input_data = np.array(np.random.random_sample(input_shape), dtype=np.float32)

Input shape: [  1 224 224   3]
Input type: <class 'numpy.float32'>
Output shape: [1 1]
Output type: <class 'numpy.float32'>


In [97]:
import cv2

In [98]:
def process_image(img_path, interpreter, input_details, output_details, input_shape, threshold=0.5):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    resized_img = cv2.resize(img, (input_shape[1], input_shape[2]))  # Resize to the model's input shape
    input_data = np.expand_dims(resized_img, axis=0).astype(np.float32)
    input_data /= 255.0  # Normalize pixel values to [0, 1]

    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])

    predicted_class = 'Pure' if output_data[0][0] > threshold else 'Adulterated'
    confidence_level = (output_data[0][0] - 0.5) * 2 * 100 if output_data[0][0] > threshold else (0.5 - output_data[0][0]) * 2 * 100
    return predicted_class, confidence_level, output_data[0][0]

In [117]:
folder_path = 'new/Pure4g/'
pure_confidence_levels = []
adulterated_confidence_levels = []

for img_name in os.listdir(folder_path):
    img_path = os.path.join(folder_path, img_name)
    if img_path.endswith(('.png', '.jpg', '.jpeg')):  # Add more extensions if needed
        predicted_class, confidence_level, output_value = process_image(img_path, interpreter, input_details, output_details, input_shape)
        if predicted_class == 'Pure':
            pure_confidence_levels.append(confidence_level)
        else:
            adulterated_confidence_levels.append(confidence_level)
        print(f'Image: {img_name}, Predicted Class: {predicted_class}, Confidence Level: {confidence_level:.2f}%, Actual Number Predicted: {output_value:.2f}')

# Calculate the average confidence levels
average_pure_confidence = sum(pure_confidence_levels) / len(pure_confidence_levels) if pure_confidence_levels else 0
average_adulterated_confidence = sum(adulterated_confidence_levels) / len(adulterated_confidence_levels) if adulterated_confidence_levels else 0

print(f'Average Confidence Level for Pure: {average_pure_confidence:.2f}%')
print(f'Average Confidence Level for Adulterated: {average_adulterated_confidence:.2f}%')

Image: A (1).jpg, Predicted Class: Pure, Confidence Level: 99.27%, Actual Number Predicted: 1.00
Image: A (10).jpg, Predicted Class: Pure, Confidence Level: 98.91%, Actual Number Predicted: 0.99
Image: A (2).jpg, Predicted Class: Adulterated, Confidence Level: 82.03%, Actual Number Predicted: 0.09
Image: A (3).jpg, Predicted Class: Pure, Confidence Level: 99.58%, Actual Number Predicted: 1.00
Image: A (4).jpg, Predicted Class: Pure, Confidence Level: 82.35%, Actual Number Predicted: 0.91
Image: A (5).jpg, Predicted Class: Pure, Confidence Level: 60.34%, Actual Number Predicted: 0.80
Image: A (6).jpg, Predicted Class: Pure, Confidence Level: 99.62%, Actual Number Predicted: 1.00
Image: A (7).jpg, Predicted Class: Pure, Confidence Level: 95.69%, Actual Number Predicted: 0.98
Image: A (8).jpg, Predicted Class: Pure, Confidence Level: 99.84%, Actual Number Predicted: 1.00
Image: A (9).jpg, Predicted Class: Pure, Confidence Level: 99.60%, Actual Number Predicted: 1.00
Average Confidence Lev